In [1]:
import matplotlib.pyplot as plt
import math

In [2]:
import numpy as np
from sklearn.datasets import load_digits
digits = load_digits()
#check the shape of digits data
print(digits.data.shape)
#check the shape of digits target
print(digits.target.shape)
#let us use the linear regression used in the previous lab
N = digits.data.shape[0] #Number of data points
d = digits.data.shape[1] #Dimension of data points
A = digits.data
#In the following code, we create a Nx1 vector of target labels
y = 1.0*np.ones([A.shape[0],1])
for i in range(digits.target.shape[0]):
  y[i] = digits.target[i]

(1797, 64)
(1797,)


In [3]:
def evalf(x,lam):
  fval = np.linalg.norm(np.matmul(A,x) - y)
  fval = 0.5 * (fval)**2+(lam/2)*np.matmul(x.T,x)
  return (fval)

def evalg(x,lam):
  grad=(lam*x+np.matmul(A.T, np.matmul(A, x) - y))
  return grad

def evalh(x,lam):
  return np.matmul(A.T,A)+lam*np.identity(d)

def compute_B_k(s, y, B_k):
  I=np.identity(64)
  mu = 1 / np.dot(np.squeeze(np.asarray(y)), np.squeeze(np.asarray(s)))
  a1 = np.outer(np.multiply(mu, s), np.transpose(y))
  a2 = np.outer(np.multiply(mu, y), np.transpose(s))
  a = np.subtract(I, a1)
  b = np.subtract(I, a2)
  c = np.outer(np.multiply(mu, s), np.transpose(s))
  d = np.matmul(np.matmul(a, B_k), b)
  z = np.add(d, c)
  return z

In [4]:
CONSTANT_STEP_LENGTH = 3
BACKTRACKING_LINE_SEARCH = 2
EXACT_LINE_SEARCH = 1

In [5]:
def compute_D_k(x):
  mat = evalh(x)
  return np.linalg.inv(mat)

def compute_steplength_backtracking_scaled_direction(x, lam,gradf, alpha_start, rho, gamma, B_k):
  alpha = alpha_start
  p = - gradf
  D_k = B_k

  while evalf(x + alpha*np.matmul(D_k,p),lam) > evalf(x,lam) + gamma * alpha* (np.matmul(np.matrix.transpose(gradf), np.matmul(D_k,p)) ):
    alpha = alpha * rho
  return alpha

In [6]:
def find_minimizer_Newtonmethod(start_x,lam, tol, line_search_type, *args):
  x = start_x
  g_x = evalg(x,lam)
  h_x = evalh(x,lam)

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  while (np.linalg.norm(g_x) > tol):
    D_k = np.linalg.inv(evalh(x,lam))
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(x,lam, g_x, alpha_start, rho, gamma, D_k)
    elif line_search_type == CONSTANT_STEP_LENGTH:
      step_length = 1.0
    else:
      raise ValueError('Line search type unknown. Please check!')

    x = np.subtract(x, np.multiply(step_length,np.matmul(D_k, g_x)))
    k = k + 1
    g_x = evalg(x,lam)
  return x,  k

In [7]:
def find_minimizer_BFGS_scaling(start_x, lam,tol, line_search_type, *args):
  x = start_x
  g_x = evalg(x,lam)

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])

  k = 0
  B_k = np.identity(d)
  x_old = x
  while (np.linalg.norm(g_x) > tol):
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(x_old,lam, g_x, alpha_start, rho, gamma, B_k)
    elif line_search_type == CONSTANT_STEP_LENGTH:
      step_length = 1.
    else:
      raise ValueError('Line search type unknown. Please check!')
    x_new = np.subtract(x_old, np.multiply(step_length, np.matmul(B_k,g_x)))
    g_x = evalg(x_new,lam)
    s = np.subtract(x_new, x_old)
    y = np.subtract(evalg(x_new,lam), evalg(x_old,lam))
    B_k = compute_B_k(s, y, B_k)
    x_old = x_new
    k = k + 1
  return x_new, k

**1. Use Newton's method to solve problem (1), which is direct OLSLR. Use
the starting point x = 0. Report the difficulties, you encounter. Check if these difficulties remain if you use Newton’s method to solve the
 problem (2), the regularized OLSLR with
λ = 0.001 and starting point x = 0. Explain the reason for your observation. Report the values
of $x^∗_{f}$ and $x^∗_{f_λ}$.**

In [8]:
##PART:2

alpha = 0.9
rho = 0.5
gamma = 0.5
my_tol= 1e-3
lam=0.001

my_start_x =np.zeros(d)
x, k= find_minimizer_Newtonmethod(my_start_x.reshape(d,1),lam, my_tol, BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
print('for starting with',my_start_x,':\nwe have minimizer:',x ,'\nno. of iterations:',k)

for starting with [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] :
we have minimizer: [[ 0.00000000e+00]
 [ 9.69076881e-02]
 [-4.32192761e-03]
 [-7.75916338e-03]
 [ 7.49591986e-02]
 [ 1.13946582e-02]
 [-2.71293921e-02]
 [-7.34410663e-03]
 [ 9.98267906e-01]
 [-2.88089376e-02]
 [ 1.18688356e-01]
 [ 6.60922719e-02]
 [-5.57075914e-02]
 [-6.97056149e-02]
 [ 9.65844013e-02]
 [ 2.55196013e-01]
 [-7.29830665e-01]
 [ 2.42711745e-02]
 [ 7.73241543e-02]
 [-2.33008498e-02]
 [-5.64077618e-02]
 [ 5.72413971e-02]
 [-4.88675247e-02]
 [-2.62555997e-01]
 [-9.06071168e-01]
 [-1.49769638e-01]
 [ 5.64022789e-02]
 [ 8.96665318e-02]
 [ 8.39315937e-02]
 [ 9.85410046e-02]
 [ 1.69269848e-03]
 [-2.96649687e+00]
 [ 0.00000000e+00]
 [-1.54361470e-01]
 [-9.32404639e-03]
 [ 1.39497978e-01]
 [-3.69237437e-02]
 [ 5.46111772e-02]
 [-9.20425585e-03]
 [ 0.00000000e+00]
 [ 1.

$\large\textbf{Question 1.}$

$\large\textbf{Answer 1.}$

$x_f^*$ = does not found

$x_{f_{\lambda}}^*$= [9.69076881e-02]
 [-4.32192761e-03]
 [-7.75916338e-03]
 [ 7.49591986e-02]
 [ 1.13946582e-02]
 [-2.71293921e-02]
 [-7.34410663e-03]
 [ 9.98267906e-01]
 [-2.88089376e-02]
 [ 1.18688356e-01]
 [ 6.60922719e-02]
 [-5.57075914e-02]
 [-6.97056149e-02]
 [ 9.65844013e-02]
 [ 2.55196013e-01]
 [-7.29830665e-01]
 [ 2.42711745e-02]
 [ 7.73241543e-02]
 [-2.33008498e-02]
 [-5.64077618e-02]
 [ 5.72413971e-02]
 [-4.88675247e-02]
 [-2.62555997e-01]
 [-9.06071168e-01]
 [-1.49769638e-01]
 [ 5.64022789e-02]
 [ 8.96665318e-02]
 [ 8.39315937e-02]
 [ 9.85410046e-02]
 [ 1.69269848e-03]
 [-2.96649687e+00]
 [ 0.00000000e+00]
 [-1.54361470e-01]
 [-9.32404639e-03]
 [ 1.39497978e-01]
 [-3.69237437e-02]
 [ 5.46111772e-02]
 [-9.20425585e-03]
 [ 0.00000000e+00]
 [ 1.03326506e-01]
 [ 1.23983484e-01]
 [-1.37635230e-02]
 [ 5.40029021e-03]
 [ 1.31185699e-01]
 [ 5.49577814e-02]
 [ 2.24935899e-02]
 [ 7.48046264e-03]
 [ 6.17507773e-01]
 [ 2.44100619e-02]
 [ 1.42356436e-03]
 [-6.21114800e-02]
 [-2.07024630e-01]
 [-3.38510517e-02]
 [ 1.05486785e-01]
 [-1.40336481e-01]
 [-9.82347826e-01]
 [-1.14473639e-01]
 [ 2.10485229e-02]
 [-4.36083412e-02]
 [ 1.87372531e-02]
 [-6.66562216e-02]
 [ 1.19384465e-02]
 [-5.27782514e-02]]

We have successfully determined the minimum using regularized Ordinary Least Squares Linear Regression (OLSLR). However, direct OLSLR fails to converge to the minimum due to the occurrence of a non-invertible Hessian matrix (singular matrix), causing computational issues.

Regularized OLSLR resolves this problem by ensuring the Hessian matrix is positive definite. This is accomplished by introducing a regularization term, denoted by λ (lambda), multiplied by the identity matrix and added to the Hessian. Consequently, the regularization term guarantees the invertibility of the Hessian, allowing us to calculate its inverse and thereby attain the minimum of the cost function.

**2. Use the BFGS method with starting point x = 0, to solve the problem (1) and describe if you observe any
 difficulty. Check if solving the regularized problem (2) helps (use λ = 0.001 and starting point x = 0). Explain
 your observations. Report the values of  $x^∗_{f}$ and $x^∗_{f_λ}$.**

In [9]:
#PART:2

alpha = 0.9
rho = 0.5
gamma = 0.5
my_tol= 1e-5
lam=0.001
my_start_x =np.zeros(d)
x, k = find_minimizer_BFGS_scaling(my_start_x.reshape((d,1)),lam, my_tol, BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
print('for starting with',i,':\nwe have minimizer:',x ,'\nno. of iterations:',k)

for starting with 1796 :
we have minimizer: [[ 0.00000000e+00]
 [ 9.69076882e-02]
 [-4.32192766e-03]
 [-7.75916335e-03]
 [ 7.49591987e-02]
 [ 1.13946582e-02]
 [-2.71293921e-02]
 [-7.34410665e-03]
 [ 9.98267907e-01]
 [-2.88089376e-02]
 [ 1.18688356e-01]
 [ 6.60922718e-02]
 [-5.57075914e-02]
 [-6.97056149e-02]
 [ 9.65844014e-02]
 [ 2.55196013e-01]
 [-7.29830666e-01]
 [ 2.42711745e-02]
 [ 7.73241544e-02]
 [-2.33008497e-02]
 [-5.64077619e-02]
 [ 5.72413971e-02]
 [-4.88675248e-02]
 [-2.62555997e-01]
 [-9.06071169e-01]
 [-1.49769638e-01]
 [ 5.64022790e-02]
 [ 8.96665319e-02]
 [ 8.39315939e-02]
 [ 9.85410047e-02]
 [ 1.69269847e-03]
 [-2.96649688e+00]
 [ 0.00000000e+00]
 [-1.54361470e-01]
 [-9.32404637e-03]
 [ 1.39497978e-01]
 [-3.69237437e-02]
 [ 5.46111772e-02]
 [-9.20425583e-03]
 [ 0.00000000e+00]
 [ 1.03326506e-01]
 [ 1.23983484e-01]
 [-1.37635229e-02]
 [ 5.40029024e-03]
 [ 1.31185700e-01]
 [ 5.49577815e-02]
 [ 2.24935899e-02]
 [ 7.48046259e-03]
 [ 6.17507773e-01]
 [ 2.44100619e-02]
 [ 1.4

In [10]:
def evalf(x):
  fval = np.linalg.norm(np.matmul(A,x) - y)
  fval = 0.5 * (fval)**2
  return (fval)

def evalg(x):
  return np.matmul(A.T, np.matmul(A, x) - y)

def evalh(x):
  return np.matmul(A.T,A)

def compute_B_k(s, y, B_k):
  mu = 1 / np.dot(np.squeeze(np.asarray(y)), np.squeeze(np.asarray(s)))
  I = np.identity(64)
  a1 = np.outer(np.multiply(mu, s), np.transpose(y))
  a2 = np.outer(np.multiply(mu, y), np.transpose(s))
  a = np.subtract(I, a1)
  b = np.subtract(I, a2)
  c = np.outer(np.multiply(mu, s), np.transpose(s))
  d = np.matmul(np.matmul(a, B_k), b)
  z = np.add(d, c)
  return z

In [11]:
CONSTANT_STEP_LENGTH = 3
BACKTRACKING_LINE_SEARCH = 2
EXACT_LINE_SEARCH = 1

In [12]:
def compute_D_k(x):
  mat = evalh(x)
  return np.linalg.inv(mat)

def compute_steplength_backtracking_scaled_direction(x, gradf, alpha_start, rho, gamma, B_k):
  n = len(x)
  alpha = alpha_start
  p = - gradf
  D_k = B_k
  while evalf(x + alpha*np.matmul(D_k,p)) > evalf(x) + gamma * alpha* (np.matmul(np.matrix.transpose(gradf), np.matmul(D_k,p)) ):
    alpha = alpha * rho
  return alpha

In [13]:
def find_minimizer_Newtonmethod(start_x, tol, line_search_type, *args):
  x = start_x
  g_x = evalg(x)
  h_x = evalh(x)

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  step_length = 1.
  while (np.linalg.norm(g_x) > tol):
    D_k = np.linalg.inv(evalh(x))
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(x, g_x, alpha_start, rho, gamma, D_k)
    elif line_search_type == CONSTANT_STEP_LENGTH:
      step_length = 1.
    else:
      raise ValueError('Line search type unknown. Please check!')
    x = np.subtract(x, np.multiply(step_length,np.matmul(D_k, g_x)))
    k += 1
    g_x = evalg(x)
  return x,  k

In [14]:
def find_minimizer_BFGS_scaling(start_x, tol, line_search_type, *args):

  x = start_x
  g_x = evalg(x)

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])

  k = 0
  B_k = np.identity(d)
  x_old = x
  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(x_old, g_x, alpha_start, rho, gamma, B_k)
    elif line_search_type == CONSTANT_STEP_LENGTH: #do a gradient descent with constant step length
      step_length = 1.
    else:
      raise ValueError('Line search type unknown. Please check!')
    x_new = np.subtract(x_old, np.multiply(step_length, np.matmul(B_k,g_x)))
    g_x = evalg(x_new)
    s = np.subtract(x_new, x_old)
    y = np.subtract(evalg(x_new), evalg(x_old))
    B_k = compute_B_k(s, y, B_k)
    x_old = x_new
    k += 1
  return x_new, k

**1. Use Newton's method to solve problem (1), which is direct OLSLR. Use
the starting point x = 0. Report the difficulties, you encounter. Check if these difficulties remain if you use Newton’s method to solve the
 problem (2), the regularized OLSLR with
λ = 0.001 and starting point x = 0. Explain the reason for your observation. Report the values
of $x^∗_{f}$ and $x^∗_{f_λ}$.**

In [15]:
#PART:1
alpha = 0.9
rho = 0.5
gamma = 0.5
my_tol= 1e-5
my_start_x =np.zeros(d)
x, k= find_minimizer_Newtonmethod(my_start_x.reshape(d,1), my_tol, BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
print('Values of x star:\n',x,'\nk=', k)

LinAlgError: Singular matrix

*we* got singular matrix which inverse is not possible.So that the dierct method OLSLR doen work.

**2. Use the BFGS method with starting point x = 0, to solve the problem (1) and describe if you observe any
 difficulty. Check if solving the regularized problem (2) helps (use λ = 0.001 and starting point x = 0). Explain
 your observations. Report the values of  $x^∗_{f}$ and $x^∗_{f_λ}$.**

In [16]:
#PART:1

alpha = 0.9
rho = 0.5
gamma = 0.5
my_tol= 1e-5
my_start_x =np.zeros(d)
x, k= find_minimizer_BFGS_scaling(my_start_x.reshape(d,1), my_tol, BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
print('Values of x_star:\n',x,'\nk=', k)

Values of x_star:
 [[ 0.00000000e+00]
 [ 9.69033568e-02]
 [-4.32277232e-03]
 [-7.76028319e-03]
 [ 7.49594380e-02]
 [ 1.13947198e-02]
 [-2.71328245e-02]
 [-7.33176333e-03]
 [ 9.98337968e-01]
 [-2.88095538e-02]
 [ 1.18688288e-01]
 [ 6.60916265e-02]
 [-5.57069862e-02]
 [-6.97063705e-02]
 [ 9.65876439e-02]
 [ 2.55182251e-01]
 [-7.29828608e-01]
 [ 2.42709916e-02]
 [ 7.73249597e-02]
 [-2.33000278e-02]
 [-5.64086144e-02]
 [ 5.72426822e-02]
 [-4.88717684e-02]
 [-2.62467763e-01]
 [-9.06562829e-01]
 [-1.49767791e-01]
 [ 5.64019538e-02]
 [ 8.96663590e-02]
 [ 8.39318159e-02]
 [ 9.85411936e-02]
 [ 1.69317614e-03]
 [-2.96805758e+00]
 [ 0.00000000e+00]
 [-1.54362338e-01]
 [-9.32361205e-03]
 [ 1.39497628e-01]
 [-3.69234835e-02]
 [ 5.46111776e-02]
 [-9.20505070e-03]
 [ 0.00000000e+00]
 [ 1.03279535e-01]
 [ 1.23983258e-01]
 [-1.37639605e-02]
 [ 5.40087816e-03]
 [ 1.31185107e-01]
 [ 5.49570758e-02]
 [ 2.24938237e-02]
 [ 7.47977909e-03]
 [ 6.17755030e-01]
 [ 2.44122357e-02]
 [ 1.42333037e-03]
 [-6.2111076

**ANS:2** \
Upon employing the Regularization OLSLR method with λ = 0.1, we observed that it required 114 iterations to converge. Subsequently, when we applied the DIRECT OLSLR method, it also converged in 114 iterations. It appears that the similarity in iteration counts may be attributed to the fact that in DIRECT OLSLR, we effectively set λ = 0, whereas in Regularization OLSLR, we used λ = 0.1. Thus, the adjustment of the regularization parameter λ resulted in achieving the same number of iterations in both methods.

